In [1]:
import pandas as pd 
import numpy
import pdfplumber
import re
import spacy

In [2]:
###put the path of skill library u have downloaded

###if facing error try to install pdfplumber first by commanding %pip install pdfplumber

lib_path=r"C:\Users\wwwja\Downloads\archive\skills.csv"


###your resume path

pdf_path = r"C:\Users\wwwja\Downloads\Ankush_Jaiswal_resume.pdf"


#### Put the job dDescription in below cell with three coloun to convert into string
Description="""Highly focused individual with self-driven attitude
Problem-solving and logical thinking to automate and improve internal processes
Using various tools such as SQL and Python for managing the various requirements for different data asset projects
Ability to diligently involve in activities like Data Cleaning, Retrieval, Manipulation, Analytics, and Reporting
Using data science and statistical techniques to build machine learning models and deal with textual data
Keep up-to-date knowledge of the industry and related markets
Ability to multitask, prioritize, and manage time efficiently
Understand needs of the hiring organization or client in order to target solutions to their benefit
Advanced speaking and writing skills for effective communication
Ability to work in cross-functional teams demonstrating high level of commitment and coordination
Attention to details and commitment to accuracy for the desired deliverable
Should demonstrate and develop a sense of ownership toward the assigned task
Ability to keep sensitive business information confidential
Contribute, positively and extensively towards building the organizational reputation, brand, and operational excellence


Desired Qualifications for Associate Data Scientist:



3-6 years of relevant experience in data science
Advanced knowledge of statistics and the basics of machine learning
Experience in dealing with textual data and using natural language processing techniques
Ability to conduct analysis to extract actionable insights
Technical skills in Python (Numpy, Pandas, NLTK, transformers, Spacy), SQL, and other programming languages for dealing with large datasets
Experience in data cleaning, manipulation, feature engineering, and building models
Experience in the end-to-end development of a data science project
Strong interpersonal skills and extremely resourceful
Proven ability to complete assigned tasks according to the outlined scope and timeline
Good language, communication, and writing skills in English
Expertise in using tools like MS Office, PowerPoint, Excel, and Word
Graduate or Post-graduate from a reputed college or university

"""


# Run everything


with pdfplumber.open(pdf_path) as pdf:
    text = ""
    for page in pdf.pages:
        text += page.extract_text() + "\n"

print(text)
##this print will return whether your resume has been parsed properly by the ppdfplumber


Ankush Kumar Jaiswal
In Progress: 2nd year M.B.A, Business Analytics
Amity University Online, Noida
Phone : +91 - 9354895519
Email : jaiswalank1999@gmail.com
Linkedin : https://www.linkedin.com/in/ankush-kumar-jaiswal-196937200
Github : https://github.com/Ankush-Kumar-Jaiswal
Address : Sangam Vihar, New Delhi – 110080
----------------------------------------------------------------------------------------
Professional Summary
Results-driven Business Analyst with a strong foundation in sales, business operations, and
business process modelling, currently pursuing an MBA in Business Analytics. Skilled in data
analysis, visualization, and predictive modeling, with hands-on experience in Python, SQL,
Power BI, and Tableau. Demonstrated ability to deliver actionable insights through real-world
projects in sentiment analysis, event analytics, and price prediction. Adept at bridging
business needs and technical solutions, optimizing processes, and engaging key stakeholders
to drive growth and

In [ ]:
##run the function

In [6]:
df=pd.read_csv(lib_path)
df["Skill"]=df["Skill"].str.replace("\n", "", regex=True)
Known_skills=df["Skill"].to_list()

# Read the PDF text
def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

# Extract name (assume first line is name if no header)
def extract_name(text):
    first_line = text.strip().split('\n')[0]
    return first_line

# Extract email
def extract_email(text):
    match = re.findall(r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+", text)
    return list(match)[0]

# Extract phone number
def extract_phone(text):
    pattern = r"(?:\+?\d{1,3}[-.\s]?)?(?:\(?\d{3,4}\)?[-.\s]?)?\d{3,4}[-.\s]?\d{4}"
    match = re.findall(pattern, text)
    phones = ["".join(m) for m in match]
    return list(set(phones))[1]

# Extract LinkedIn
def extract_linkedin(text):
    match = re.findall(r"(https?://(www\.)?linkedin\.com/in/[A-Za-z0-9_-]+)", text)
    return [m[0] for m in match][0]

# Extract location (very approximate: look for words like city, state or pin)
def extract_location(text):
    # you may replace this with your list of cities or patterns
    pattern = r"(?:\b[A-Z][a-z]*(?:\s+[A-Z][a-z]*)*(?:,)?\s*)+[–-]?\s*\d{6}"
    match = re.findall(pattern, text)

    if match:
        return match[0]
    return []

# Extract education (look for degrees)
def extract_education(text):
    education_keywords = [
    "Bachelor", "Bachelors", "Bachelor's",
    "Master", "Masters", "Master's",
    "B\.Sc", "M\.Sc", "MBA", "PhD", "Diploma", "B\.Tech", "M\.Tech"
]


    lines = text.split('\n')
    education = []
    in_education_section = False

    for i, line in enumerate(lines):
        line_lower = line.strip().lower()

        # Look for education section header
        if re.search(r'education|academic background|educational qualifications', line_lower):
            in_education_section = True
            continue

        # Stop if we reach another section
        if in_education_section and re.search(r'(experience|skills|projects|certifications|summary|profile)', line_lower):
            break

        # If inside education section, look for degrees
        if in_education_section:
            for k in education_keywords:
                if re.search(k, line, re.IGNORECASE):
                    education.append(line.strip())
                    break

    return education

def extract_skills(text, known_skills):
    """
    Extract skills from full text after cleaning it.
    Matches only known skills.
    """
    found_skills = set()

    # Lowercase & clean text
    text_clean = text.lower()
    # Remove unwanted characters: (),., commas, etc.
    text_clean = re.sub(r'[(),.]', ' ', text_clean)
    # Replace multiple spaces with single space
    text_clean = re.sub(r'\s+', ' ', text_clean)

    for skill in known_skills:
        # Match skill in cleaned text
        if re.search(rf'\b{re.escape(skill.lower())}\b', text_clean):
            found_skills.add(skill)

    found_skills= list(dict.fromkeys(found_skills))


    return sorted(found_skills)



def extract_certifications(text):
    certifications = []
    lines = text.split('\n')
    in_cert_section = False

    for line in lines:
        line_lower = line.strip().lower()

        # Detect certifications section header
        if re.search(r'(certifications?|licenses|accreditations)', line_lower):
            in_cert_section = True
            continue

        # Stop at next section header
        if in_cert_section and re.search(
            r'(experience|education|skills|projects|summary|profile|work|achievements)', 
            line_lower
        ):
            break

        # If inside certifications section, and line is not empty
        if in_cert_section:
            if line.strip():
                certifications.append(line.strip())

    return certifications




text = extract_text_from_pdf(pdf_path)

data = {
    "name": extract_name(text),
    "email": extract_email(text),
    "phone": extract_phone(text),
    "linkedin": extract_linkedin(text),
    "location": extract_location(text),
    "education": extract_education(text),
    "skills": extract_skills(text, Known_skills), 
    "certification": extract_certifications(text)
}

def extract_skills_from_jd(text, known_skills):
    """
    Extract key skills mentioned in a LinkedIn job description.

    :param text: The job description text (string)
    :param known_skills: (optional) a predefined list of skills to match against
    :return: list of detected skills
    """
    # Lowercase the text for uniformity
    text_lower = text.lower()


    found_skills = set()

    for skill in known_skills:
        # use word boundaries to avoid partial matches
        if re.search(rf'\b{re.escape(skill)}\b', text_lower):
            found_skills.add(skill)
    found_skills=list(dict.fromkeys(found_skills))


    return sorted(found_skills)

def report(ans, data):
    not_matched_skills=[]
    count=0

    for i in ans:
        if i in data["skills"]:
            count += 1

        else:
            not_matched_skills.append(i)
    Percent_job_match=(count/len(ans))*100
    return Percent_job_match, not_matched_skills
ans=extract_skills_from_jd(Description, Known_skills)
percent, not_match=report(ans, data)
print(f"Name:{data["name"]}")
print(f"E-mail Id:{data["email"]}")
print(f"Linkedin :{data["linkedin"]}")

print(f"Percentage of job Match: {percent}%")
print(f"Skills That Do Not Match: {", ".join(not_match)}")


Name:Ankush Kumar Jaiswal
E-mail Id:jaiswalank1999@gmail.com
Linkedin :https://www.linkedin.com/in/ankush-kumar-jaiswal-196937200
Percentage of job Match: 48.07692307692308%
Skills That Do Not Match: 3, 6, building, building models, business information, communication, contribute, english, hiring, interpersonal skills, language processing, languages, natural language, natural language processing, nltk, operational excellence, organization, ownership, powerpoint, processing, programming languages, project, reporting, spacy, statistics, word, writing
